<a href="https://colab.research.google.com/github/Koppurapu/MAIN/blob/main/IRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


In [ ]:
# Required packages
# pip install numpy opencv-python scikit-learn pillow matplotlib scipy scikit-image

In [ ]:
#module_1:Create Feature Extractor Class
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops

class FeatureExtractor:
    def __init__(self):
        self.feature_dim = 0

    def extract_color_histogram(self, image, bins=32):
        """Extract color histogram in HSV space"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hist_h = cv2.calcHist([hsv], [0], None, [bins], [0, 180])
        hist_s = cv2.calcHist([hsv], [1], None, [bins], [0, 256])
        hist_v = cv2.calcHist([hsv], [2], None, [bins], [0, 256])

        # Normalize
        hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_color_moments(self, image):
        """Extract color moments (mean, std, skewness)"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        moments = []
        for channel in cv2.split(hsv):
            moments.append(np.mean(channel))
            moments.append(np.std(channel))
            moments.append(self._skewness(channel))
        return np.array(moments)

    def _skewness(self, channel):
        mean = np.mean(channel)
        std = np.std(channel)
        if std == 0:
            return 0
        return np.mean(((channel - mean) / std) ** 3)

    def extract_texture_lbp(self, image, P=8, R=1):
        """Extract Local Binary Pattern texture features"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, P, R, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=P+2, range=(0, P+2))
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_texture_glcm(self, image):
        """Extract GLCM texture features"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Quantize to reduce computation
        gray = (gray / 16).astype(np.uint8)

        glcm = graycomatrix(gray, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],
                           levels=16, symmetric=True, normed=True)

        # Extract properties
        contrast = graycoprops(glcm, 'contrast').flatten()
        dissimilarity = graycoprops(glcm, 'dissimilarity').flatten()
        homogeneity = graycoprops(glcm, 'homogeneity').flatten()
        energy = graycoprops(glcm, 'energy').flatten()
        correlation = graycoprops(glcm, 'correlation').flatten()

        return np.concatenate([contrast, dissimilarity, homogeneity,
                              energy, correlation])

    def extract_edge_histogram(self, image, bins=32):
        """Extract edge orientation histogram"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Sobel gradients
        gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        magnitude = np.sqrt(gx**2 + gy**2)
        orientation = np.arctan2(gy, gx)

        # Create histogram of edge orientations
        hist, _ = np.histogram(orientation[magnitude > np.percentile(magnitude, 75)],
                              bins=bins, range=(-np.pi, np.pi))
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_features(self, image):
        """Combine all features"""
        color_hist = self.extract_color_histogram(image, bins=32)
        color_moments = self.extract_color_moments(image)
        texture_lbp = self.extract_texture_lbp(image)
        texture_glcm = self.extract_texture_glcm(image)
        edge_hist = self.extract_edge_histogram(image, bins=16)

        features = np.concatenate([
            color_hist,
            color_moments,
            texture_lbp,
            texture_glcm,
            edge_hist
        ])

        self.feature_dim = len(features)
        return features

In [ ]:
#module_1.2: Extract Features from Dataset
import os
from tqdm import tqdm

def extract_dataset_features(image_folder, output_file='features.npz'):
    """Extract features from all images in dataset"""
    extractor = FeatureExtractor()

    image_files = [f for f in os.listdir(image_folder)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    features_list = []
    filenames = []

    print("Extracting features from images...")
    for img_file in tqdm(image_files):
        img_path = os.path.join(image_folder, img_file)
        image = cv2.imread(img_path)

        if image is None:
            continue

        # Resize for consistency
        image = cv2.resize(image, (256, 256))

        features = extractor.extract_features(image)
        features_list.append(features)
        filenames.append(img_file)

    features_array = np.array(features_list)

    # Normalize features
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features_array)

    # Save
    np.savez(output_file,
             features=features_normalized,
             filenames=filenames,
             scaler_mean=scaler.mean_,
             scaler_scale=scaler.scale_)

    print(f"Extracted {len(features_list)} feature vectors")
    print(f"Feature dimension: {features_array.shape[1]}")

    return features_normalized, filenames, scaler

In [ ]:
#module_2:SVM-Based Retrieval System
#2:Create Retrieval Engine
from sklearn.svm import SVC
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

class SVMRetrievalSystem:
    def __init__(self, features, filenames, image_folder):
        self.features = features
        self.filenames = filenames
        self.image_folder = image_folder
        self.n_images = len(filenames)

        # SVM model
        self.svm = None
        self.is_trained = False

        # Feedback history
        self.relevant_indices = []
        self.irrelevant_indices = []
        self.feedback_rounds = 0

    def initial_retrieval(self, query_idx, top_k=20, metric='euclidean'):
        """Initial retrieval based on distance metric"""
        query_features = self.features[query_idx].reshape(1, -1)

        if metric == 'euclidean':
            distances = euclidean_distances(query_features, self.features)[0]
            # Lower distance = more similar
            scores = -distances
        elif metric == 'cosine':
            similarities = cosine_similarity(query_features, self.features)[0]
            scores = similarities

        # Sort by scores (descending)
        ranked_indices = np.argsort(scores)[::-1]

        # Exclude query image itself
        ranked_indices = ranked_indices[ranked_indices != query_idx]

        return ranked_indices[:top_k], scores[ranked_indices[:top_k]]

    def train_svm(self):
        """Train SVM on feedback"""
        if len(self.relevant_indices) == 0:
            print("No relevant feedback provided")
            return False

        # Prepare training data
        X_train = []
        y_train = []

        # Positive examples
        for idx in self.relevant_indices:
            X_train.append(self.features[idx])
            y_train.append(1)

        # Negative examples
        for idx in self.irrelevant_indices:
            X_train.append(self.features[idx])
            y_train.append(-1)

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        # Handle class imbalance
        n_pos = np.sum(y_train == 1)
        n_neg = np.sum(y_train == -1)

        if n_neg == 0:
            # Need negative examples - use farthest images from positives
            pos_features = self.features[self.relevant_indices]
            pos_mean = np.mean(pos_features, axis=0, keepdims=True)
            distances = euclidean_distances(pos_mean, self.features)[0]

            # Add farthest images as pseudo-negatives
            pseudo_neg_indices = np.argsort(distances)[-max(n_pos, 10):]
            for idx in pseudo_neg_indices:
                if idx not in self.relevant_indices:
                    X_train = np.vstack([X_train, self.features[idx]])
                    y_train = np.append(y_train, -1)

            n_neg = np.sum(y_train == -1)

        # Set class weights
        class_weight = {1: n_neg / n_pos, -1: 1.0} if n_pos > 0 else None

        # Train SVM with RBF kernel
        self.svm = SVC(kernel='rbf', C=1.0, gamma='scale',
                      class_weight=class_weight, probability=True)

        self.svm.fit(X_train, y_train)
        self.is_trained = True

        return True

    def svm_retrieval(self, top_k=20):
        """Retrieve using trained SVM"""
        if not self.is_trained:
            print("SVM not trained yet")
            return None, None

        # Get decision function values (distance from hyperplane)
        decision_values = self.svm.decision_function(self.features)

        # Rank by decision values (higher = more relevant)
        ranked_indices = np.argsort(decision_values)[::-1]

        # Filter out already labeled images
        labeled_indices = set(self.relevant_indices + self.irrelevant_indices)
        ranked_indices = [idx for idx in ranked_indices
                         if idx not in labeled_indices]

        return ranked_indices[:top_k], decision_values[ranked_indices[:top_k]]

    def add_feedback(self, relevant_list, irrelevant_list):
        """Add user feedback"""
        self.relevant_indices.extend(relevant_list)
        self.irrelevant_indices.extend(irrelevant_list)
        self.feedback_rounds += 1

        print(f"Feedback round {self.feedback_rounds}:")
        print(f"  Total relevant: {len(self.relevant_indices)}")
        print(f"  Total irrelevant: {len(self.irrelevant_indices)}")

    def compute_precision(self, retrieved_indices, ground_truth_relevant):
        """Compute precision metric"""
        retrieved_set = set(retrieved_indices)
        relevant_set = set(ground_truth_relevant)

        true_positives = len(retrieved_set.intersection(relevant_set))
        precision = true_positives / len(retrieved_indices) if len(retrieved_indices) > 0 else 0

        return precision

In [ ]:
#2.1:: Active Learning Component
class ActiveLearningSelector:
    """Select most informative images for user feedback"""

    @staticmethod
    def uncertainty_sampling(svm_model, features, n_select=10):
        """Select images closest to decision boundary"""
        decision_values = svm_model.decision_function(features)
        uncertainties = np.abs(decision_values)

        # Sort by uncertainty (lowest first)
        uncertain_indices = np.argsort(uncertainties)

        return uncertain_indices[:n_select]

    @staticmethod
    def diversity_sampling(features, candidate_indices, n_select=10):
        """Select diverse images using k-means"""
        from sklearn.cluster import KMeans

        candidate_features = features[candidate_indices]

        if len(candidate_indices) <= n_select:
            return candidate_indices

        kmeans = KMeans(n_clusters=n_select, random_state=42)
        kmeans.fit(candidate_features)

        # Select image closest to each cluster center
        selected = []
        for i in range(n_select):
            center = kmeans.cluster_centers_[i]
            cluster_mask = kmeans.labels_ == i
            cluster_indices = np.where(cluster_mask)[0]

            if len(cluster_indices) > 0:
                distances = euclidean_distances(
                    center.reshape(1, -1),
                    candidate_features[cluster_indices]
                )[0]
                closest_in_cluster = cluster_indices[np.argmin(distances)]
                selected.append(candidate_indices[closest_in_cluster])

        return selected

In [ ]:
#module_3:User Interface
# Create Interactive GUI
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import matplotlib.patches as mpatches

class RetrievalGUI:
    def __init__(self, retrieval_system):
        self.system = retrieval_system
        self.current_results = []
        self.current_scores = []
        self.selected_relevant = []
        self.selected_irrelevant = []

    def display_results(self, indices, scores, title="Retrieval Results"):
        """Display retrieval results in grid"""
        n_images = len(indices)
        n_cols = 5
        n_rows = (n_images + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
        fig.suptitle(title, fontsize=16)

        axes = axes.flatten() if n_images > 1 else [axes]

        for i, (idx, score) in enumerate(zip(indices, scores)):
            if i >= len(axes):
                break

            img_path = os.path.join(self.system.image_folder,
                                   self.system.filenames[idx])
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            axes[i].imshow(img_rgb)
            axes[i].set_title(f"Score: {score:.3f}", fontsize=8)
            axes[i].axis('off')

            # Add colored border for selected images
            if idx in self.selected_relevant:
                rect = mpatches.Rectangle((0, 0), img_rgb.shape[1]-1,
                                         img_rgb.shape[0]-1,
                                         linewidth=5, edgecolor='green',
                                         facecolor='none')
                axes[i].add_patch(rect)
            elif idx in self.selected_irrelevant:
                rect = mpatches.Rectangle((0, 0), img_rgb.shape[1]-1,
                                         img_rgb.shape[0]-1,
                                         linewidth=5, edgecolor='red',
                                         facecolor='none')
                axes[i].add_patch(rect)

        # Hide empty subplots
        for i in range(len(indices), len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

    def run_interactive_session(self, query_idx, max_rounds=5):
        """Run interactive retrieval session"""
        print("=" * 60)
        print("SVM-Based Image Retrieval with Relevance Feedback")
        print("=" * 60)

        # Initial retrieval
        print("\n[Round 0] Initial Retrieval...")
        results, scores = self.system.initial_retrieval(query_idx, top_k=20)

        self.display_results(results, scores,
                           title="Initial Retrieval (Round 0)")

        # Feedback rounds
        for round_num in range(1, max_rounds + 1):
            print(f"\n[Round {round_num}] Collecting Feedback...")
            print("Enter relevant image indices (comma-separated): ", end="")
            relevant_input = input().strip()

            print("Enter irrelevant image indices (comma-separated): ", end="")
            irrelevant_input = input().strip()

            # Parse input
            relevant_list = []
            if relevant_input:
                relevant_list = [results[int(i)] for i in relevant_input.split(',')
                               if i.strip().isdigit() and int(i) < len(results)]

            irrelevant_list = []
            if irrelevant_input:
                irrelevant_list = [results[int(i)] for i in irrelevant_input.split(',')
                                 if i.strip().isdigit() and int(i) < len(results)]

            if not relevant_list and not irrelevant_list:
                print("No feedback provided. Ending session.")
                break

            # Add feedback
            self.system.add_feedback(relevant_list, irrelevant_list)

            # Train SVM
            print(f"\n[Round {round_num}] Training SVM...")
            success = self.system.train_svm()

            if not success:
                print("Training failed. Ending session.")
                break

            # SVM-based retrieval
            print(f"[Round {round_num}] Retrieving with SVM...")
            results, scores = self.system.svm_retrieval(top_k=20)

            if results is None:
                break

            self.display_results(results, scores,
                               title=f"SVM Retrieval (Round {round_num})")

            # Ask to continue
            print(f"\nContinue to round {round_num + 1}? (y/n): ", end="")
            continue_input = input().strip().lower()

            if continue_input != 'y':
                print("Session ended by user.")
                break

        print("\n" + "=" * 60)
        print("Retrieval session completed!")
        print("=" * 60)

In [ ]:
#3.1:Simplified Command-Line Interface
def run_cli_session(retrieval_system, query_idx, max_rounds=5):
    """Simplified command-line interface"""

    print("\n" + "="*60)
    print("SVM Image Retrieval - Multi-Round Feedback")
    print("="*60)

    # Round 0: Initial retrieval
    print("\n>>> Round 0: Initial Retrieval")
    results, scores = retrieval_system.initial_retrieval(query_idx, top_k=20)

    print("\nTop 20 results:")
    for i, (idx, score) in enumerate(zip(results[:20], scores[:20])):
        print(f"  [{i}] {retrieval_system.filenames[idx]} (score: {score:.4f})")

    # Feedback rounds
    for round_num in range(1, max_rounds + 1):
        print(f"\n>>> Round {round_num}: Provide Feedback")
        print("Enter indices of RELEVANT images (e.g., 0,3,5): ", end="")
        rel_input = input().strip()

        print("Enter indices of IRRELEVANT images (e.g., 1,2,4): ", end="")
        irrel_input = input().strip()

        relevant_list = []
        if rel_input:
            relevant_list = [results[int(i)] for i in rel_input.split(',')
                           if i.strip().isdigit() and 0 <= int(i) < len(results)]

        irrelevant_list = []
        if irrel_input:
            irrelevant_list = [results[int(i)] for i in irrel_input.split(',')
                             if i.strip().isdigit() and 0 <= int(i) < len(results)]

        if not relevant_list:
            print("Warning: No relevant feedback provided!")
            break

        # Update system
        retrieval_system.add_feedback(relevant_list, irrelevant_list)

        # Train and retrieve
        print(f"\nTraining SVM with {len(relevant_list)} relevant and {len(irrelevant_list)} irrelevant examples...")
        retrieval_system.train_svm()

        results, scores = retrieval_system.svm_retrieval(top_k=20)

        print(f"\nUpdated top 20 results:")
        for i, (idx, score) in enumerate(zip(results[:20], scores[:20])):
            marker = "✓" if idx in retrieval_system.relevant_indices else ""
            print(f"  [{i}] {retrieval_system.filenames[idx]} (score: {score:.4f}) {marker}")

        print(f"\nContinue? (y/n): ", end="")
        if input().strip().lower() != 'y':
            break

    print("\n" + "="*60)
    print("Session complete!")

In [ ]:
#module_4: Main Execution Script
#Complete Main Script
def main():
    # Configuration
    IMAGE_FOLDER = ""
    FEATURES_FILE = "features.npz"

    # Step 1: Extract features (run once)
    if not os.path.exists(FEATURES_FILE):
        print("Extracting features from dataset...")
        features, filenames, scaler = extract_dataset_features(
            IMAGE_FOLDER, FEATURES_FILE
        )
    else:
        print("Loading pre-computed features...")
        data = np.load(FEATURES_FILE)
        features = data['features']
        filenames = data['filenames']

    print(f"Loaded {len(features)} images")
    print(f"Feature dimension: {features.shape[1]}")

    # Step 2: Initialize retrieval system
    retrieval_system = SVMRetrievalSystem(features, filenames, IMAGE_FOLDER)

    # Step 3: Select query image
    print("\nAvailable images:")
    for i, fname in enumerate(filenames[:20]):
        print(f"  [{i}] {fname}")
    print("  ...")

    query_idx = int(input("\nEnter query image index: "))

    # Step 4: Run retrieval session
    mode = input("Run with GUI (g) or CLI (c)? ").strip().lower()

    if mode == 'g':
        gui = RetrievalGUI(retrieval_system)
        gui.run_interactive_session(query_idx, max_rounds=5)
    else:
        run_cli_session(retrieval_system, query_idx, max_rounds=5)

if __name__ == "__main__":
    main()

In [ ]:
# ... (rest of the code before main function)

def main():
    """Main entry point"""
    import argparse

    parser = argparse.ArgumentParser(
        description='SVM-Based Image Retrieval with Multi-Round Relevance Feedback'
    )
    parser.add_argument('--dataset', default='cifar10_images',
                       help='Path to image dataset folder')
    parser.add_argument('--features', default='features.npz',
                       help='Path to save/load features')
    parser.add_argument('--extract', action='store_true',
                       help='Force feature extraction even if features file exists')
    parser.add_argument('--query-idx', type=int, default=None,
                       help='Query image index (if not provided, will prompt)')
    parser.add_argument('--mode', choices=['interactive', 'evaluate', 'demo'],
                       default='interactive',
                       help='Running mode')
    parser.add_argument('--max-rounds', type=int, default=5,
                       help='Maximum number of feedback rounds')
    parser.add_argument('--visualize', action='store_true',
                       help='Show visual results (requires matplotlib)')

    args = parser.parse_args([]) # Changed to parse an empty list

    print("\n" + "="*70)
    print("SVM-BASED IMAGE RETRIEVAL SYSTEM")
    print("Multi-Round Relevance Feedback")
    print("="*70)

    # ... (rest of the main function code)


In [ ]:
import os

# Create the dataset folder if it doesn't exist
IMAGE_FOLDER = 'cifar10_images'
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)
    print(f"Created empty folder: {IMAGE_FOLDER}")
else:
    print(f"Folder already exists: {IMAGE_FOLDER}")

# Now, call the main function from the previously defined cell.
# Make sure all necessary classes and functions (FeatureExtractor, SVMRetrievalSystem, etc.)
# are defined in cell 0_7sdRErqCgl (or earlier executed cells).
main()

Created empty folder: cifar10_images

SVM-BASED IMAGE RETRIEVAL SYSTEM
Multi-Round Relevance Feedback


In [ ]:
import numpy as np
import cv2
import os
from tqdm import tqdm

# Define CIFAR-10 class names
cifar10_class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# Base folder for saving images
IMAGE_FOLDER = 'cifar10_images'

def save_cifar10_images(images, labels, dataset_type, base_folder):
    print(f"Saving {dataset_type} images...")
    for i in tqdm(range(len(images))):
        img = images[i]
        label = labels[i][0]  # Labels are in a nested array
        class_name = cifar10_class_names[label]

        # Create class-specific directory if it doesn't exist
        class_folder = os.path.join(base_folder, class_name)
        os.makedirs(class_folder, exist_ok=True)

        # Save the image
        img_path = os.path.join(class_folder, f'{dataset_type}_{i:05d}.png')
        cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# Assuming x_train, y_train, x_test, y_test are already loaded
# from tensorflow.keras.datasets import cifar10
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

save_cifar10_images(x_train, y_train, 'train', IMAGE_FOLDER)
save_cifar10_images(x_test, y_test, 'test', IMAGE_FOLDER)

print(f"All CIFAR-10 images saved to '{IMAGE_FOLDER}' directory.")

Saving train images...


100%|██████████| 50000/50000 [00:14<00:00, 3520.10it/s]


Saving test images...


100%|██████████| 10000/10000 [00:02<00:00, 3372.13it/s]

All CIFAR-10 images saved to 'cifar10_images' directory.


In [ ]:
class FeatureExtractor:
    """Extract low-level features from images"""

    def __init__(self):
        self.feature_dim = 0

    def extract_color_histogram(self, image, bins=32):
        """Extract color histogram in HSV space"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hist_h = cv2.calcHist([hsv], [0], None, [bins], [0, 180])
        hist_s = cv2.calcHist([hsv], [1], None, [bins], [0, 256])
        hist_v = cv2.calcHist([hsv], [2], None, [bins], [0, 256])

        hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_color_moments(self, image):
        """Extract color moments (mean, std, skewness)"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        moments = []
        for channel in cv2.split(hsv):
            moments.append(np.mean(channel))
            moments.append(np.std(channel))
            moments.append(self._skewness(channel))
        return np.array(moments)

    def _skewness(self, channel):
        mean = np.mean(channel)
        std = np.std(channel)
        if std == 0:
            return 0
        return np.mean(((channel - mean) / std) ** 3)

    def extract_texture_lbp(self, image, P=8, R=1):
        """Extract Local Binary Pattern texture features"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, P, R, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=P+2, range=(0, P+2))
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_edge_histogram(self, image, bins=16):
        """Extract edge orientation histogram"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

        magnitude = np.sqrt(gx**2 + gy**2)
        orientation = np.arctan2(gy, gx)

        hist, _ = np.histogram(orientation[magnitude > np.percentile(magnitude, 75)],
                              bins=bins, range=(-np.pi, np.pi))
        hist = hist / (hist.sum() + 1e-7)
        return hist

    def extract_features(self, image):
        """Combine all features"""
        color_hist = self.extract_color_histogram(image, bins=32)
        color_moments = self.extract_color_moments(image)
        texture_lbp = self.extract_texture_lbp(image)
        edge_hist = self.extract_edge_histogram(image, bins=16)

        features = np.concatenate([
            color_hist,
            color_moments,
            texture_lbp,
            edge_hist
        ])

        self.feature_dim = len(features)
        return features

def extract_dataset_features(image_folder, output_file='features.npz',
                            resize_dim=(128, 128)):
    """Extract features from all images in dataset"""
    extractor = FeatureExtractor()

    # Get all image files
    image_files = []
    for root, dirs, files in os.walk(image_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                image_files.append(os.path.join(root, file))

    features_list = []
    filenames = []
    categories = []

    print(f"Extracting features from {len(image_files)} images...")
    for img_path in tqdm(image_files):
        image = cv2.imread(img_path)

        if image is None:
            continue

        # Resize for consistency
        image = cv2.resize(image, resize_dim)

        # Extract features
        features = extractor.extract_features(image)
        features_list.append(features)

        # Store relative path
        rel_path = os.path.relpath(img_path, image_folder)
        filenames.append(rel_path)

        # Extract category from path
        category = os.path.dirname(rel_path)
        categories.append(category)

    features_array = np.array(features_list)

    # Normalize features
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features_array)

    # Save
    np.savez(output_file,
             features=features_normalized,
             filenames=np.array(filenames),
             categories=np.array(categories),
             scaler_mean=scaler.mean_,
             scaler_scale=scaler.scale_)

    print(f"✓ Extracted {len(features_list)} feature vectors")
    print(f"✓ Feature dimension: {features_array.shape[1]}")
    print(f"✓ Saved to: {output_file}")

    return features_normalized, filenames, categories, scaler

In [ ]:
class SVMRetrievalSystem:
    """SVM-based image retrieval with relevance feedback"""

    def __init__(self, features, filenames, categories, image_folder):
        self.features = features
        self.filenames = filenames
        self.categories = categories
        self.image_folder = image_folder
        self.n_images = len(filenames)

        # SVM model
        self.svm = None
        self.is_trained = False

        # Feedback history
        self.relevant_indices = []
        self.irrelevant_indices = []
        self.feedback_rounds = 0

    def initial_retrieval(self, query_idx, top_k=20, metric='euclidean'):
        """Initial retrieval based on distance metric"""
        query_features = self.features[query_idx].reshape(1, -1)

        if metric == 'euclidean':
            distances = euclidean_distances(query_features, self.features)[0]
            scores = -distances  # Convert to similarity
        elif metric == 'cosine':
            scores = cosine_similarity(query_features, self.features)[0]
        else:
            raise ValueError(f"Unknown metric: {metric}")

        # Sort by scores (descending)
        ranked_indices = np.argsort(scores)[::-1]

        # Exclude query image itself
        ranked_indices = ranked_indices[ranked_indices != query_idx]

        return ranked_indices[:top_k], scores[ranked_indices[:top_k]]

    def train_svm(self, C=1.0, gamma='scale'):
        """Train SVM on feedback"""
        if len(self.relevant_indices) == 0:
            print("⚠ No relevant feedback provided")
            return False

        # Prepare training data
        X_train = []
        y_train = []

        # Positive examples
        for idx in self.relevant_indices:
            X_train.append(self.features[idx])
            y_train.append(1)

        # Negative examples
        for idx in self.irrelevant_indices:
            X_train.append(self.features[idx])
            y_train.append(-1)

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        # Handle missing negative examples
        n_pos = np.sum(y_train == 1)
        n_neg = np.sum(y_train == -1)

        if n_neg == 0:
            # Use farthest images as pseudo-negatives
            pos_features = self.features[self.relevant_indices]
            pos_mean = np.mean(pos_features, axis=0, keepdims=True)
            distances = euclidean_distances(pos_mean, self.features)[0]

            pseudo_neg_indices = np.argsort(distances)[-max(n_pos * 2, 20):]
            for idx in pseudo_neg_indices:
                if idx not in self.relevant_indices:
                    X_train = np.vstack([X_train, self.features[idx]])
                    y_train = np.append(y_train, -1)

            n_neg = np.sum(y_train == -1)
            print(f"  Added {n_neg} pseudo-negative examples")

        # Set class weights
        class_weight = {1: n_neg / (n_pos + 1e-7), -1: 1.0}

        # Train SVM
        self.svm = SVC(kernel='rbf', C=C, gamma=gamma,
                      class_weight=class_weight, probability=True)

        self.svm.fit(X_train, y_train)
        self.is_trained = True

        return True

    def svm_retrieval(self, top_k=20):
        """Retrieve using trained SVM"""
        if not self.is_trained:
            print("⚠ SVM not trained yet")
            return None, None

        # Get decision function values
        decision_values = self.svm.decision_function(self.features)

        # Rank by decision values
        ranked_indices = np.argsort(decision_values)[::-1]

        # Filter out already labeled images
        labeled_indices = set(self.relevant_indices + self.irrelevant_indices)
        ranked_indices = [idx for idx in ranked_indices
                         if idx not in labeled_indices]

        return ranked_indices[:top_k], decision_values[ranked_indices[:top_k]]

    def add_feedback(self, relevant_list, irrelevant_list):
        """Add user feedback"""
        self.relevant_indices.extend(relevant_list)
        self.irrelevant_indices.extend(irrelevant_list)
        self.feedback_rounds += 1

        print(f"\n📊 Feedback Round {self.feedback_rounds}:")
        print(f"   Relevant: {len(relevant_list)} (Total: {len(self.relevant_indices)})")
        print(f"   Irrelevant: {len(irrelevant_list)} (Total: {len(self.irrelevant_indices)})")

    def compute_precision(self, retrieved_indices, ground_truth_relevant, k=20):
        """Compute precision@k"""
        retrieved_k = retrieved_indices[:k]
        relevant_set = set(ground_truth_relevant)
        hits = sum(1 for idx in retrieved_k if idx in relevant_set)
        return hits / k if k > 0 else 0

    def get_image_path(self, idx):
        """Get full image path"""
        return os.path.join(self.image_folder, self.filenames[idx])

In [ ]:
def display_images_cli(retrieval_system, indices, scores, title="Results"):
    """Display images in CLI with scores"""
    print(f"\n{'='*70}")
    print(f"{title:^70}")
    print(f"{'='*70}")

    for i, (idx, score) in enumerate(zip(indices, scores)):
        filename = retrieval_system.filenames[idx]
        category = retrieval_system.categories[idx]

        marker = ""
        if idx in retrieval_system.relevant_indices:
            marker = " ✓ RELEVANT"
        elif idx in retrieval_system.irrelevant_indices:
            marker = " ✗ IRRELEVANT"

        print(f"[{i:2d}] Score: {score:7.4f} | {category:12s} | {filename}{marker}")

def run_cli_session(retrieval_system, query_idx, max_rounds=5):
    """Command-line interface for retrieval"""

    print("\n" + "="*70)
    print("SVM IMAGE RETRIEVAL - MULTI-ROUND RELEVANCE FEEDBACK")
    print("="*70)

    # Display query image
    query_file = retrieval_system.filenames[query_idx]
    query_category = retrieval_system.categories[query_idx]
    print(f"\n🔍 Query Image: {query_file}")
    print(f"   Category: {query_category}")

    # Round 0: Initial retrieval
    print(f"\n{'─'*70}")
    print("ROUND 0: Initial Retrieval (Distance-based)")
    print(f"{'─'*70}")

    results, scores = retrieval_system.initial_retrieval(query_idx, top_k=20)
    display_images_cli(retrieval_system, results, scores, "Top 20 Initial Results")

    # Feedback rounds
    for round_num in range(1, max_rounds + 1):
        print(f"\n{'─'*70}")
        print(f"ROUND {round_num}: Provide Feedback")
        print(f"{'─'*70}")

        print("\nEnter indices of RELEVANT images (e.g., 0,3,5 or 0-5): ", end="")
        rel_input = input().strip()

        print("Enter indices of IRRELEVANT images (e.g., 1,2,4 or 6-10): ", end="")
        irrel_input = input().strip()

        # Parse input
        def parse_indices(input_str, max_idx):
            indices = []
            if not input_str:
                return indices

            parts = input_str.replace(' ', '').split(',')
            for part in parts:
                if '-' in part:
                    start, end = map(int, part.split('-'))
                    indices.extend(range(start, min(end + 1, max_idx)))
                elif part.isdigit():
                    idx = int(part)
                    if 0 <= idx < max_idx:
                        indices.append(idx)
            return indices

        rel_positions = parse_indices(rel_input, len(results))
        irrel_positions = parse_indices(irrel_input, len(results))

        relevant_list = [results[i] for i in rel_positions]
        irrelevant_list = [results[i] for i in irrel_positions]

        if not relevant_list:
            print("\n⚠ No relevant feedback provided. Ending session.")
            break

        # Add feedback
        retrieval_system.add_feedback(relevant_list, irrelevant_list)

        # Train SVM
        print(f"\n🔧 Training SVM...")
        success = retrieval_system.train_svm()

        if not success:
            break

        # SVM retrieval
        print(f"🔍 Retrieving with SVM...")
        results, scores = retrieval_system.svm_retrieval(top_k=20)

        if results is None:
            break

        display_images_cli(retrieval_system, results, scores,
                          f"Top 20 Results (SVM Round {round_num})")

        # Ask to continue
        if round_num < max_rounds:
            print(f"\n{'─'*70}")
            print(f"Continue to Round {round_num + 1}? (y/n): ", end="")
            if input().strip().lower() != 'y':
                break

    print("\n" + "="*70)
    print("SESSION COMPLETE")
    print("="*70)

    # Summary
    print(f"\nTotal Feedback Collected:")
    print(f"  ✓ Relevant images: {len(retrieval_system.relevant_indices)}")
    print(f"  ✗ Irrelevant images: {len(retrieval_system.irrelevant_indices)}")
    print(f"  Total rounds: {retrieval_system.feedback_rounds}")

In [ ]:
def visualize_results(retrieval_system, indices, scores, title="Results",
                     save_path=None):
    """Visualize retrieval results in a grid"""
    n_images = min(len(indices), 20)
    n_cols = 5
    n_rows = (n_images + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
    fig.suptitle(title, fontsize=16, fontweight='bold')

    if n_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for i in range(n_cols * n_rows):
        if i < n_images:
            idx = indices[i]
            score = scores[i]

            img_path = retrieval_system.get_image_path(idx)
            img = cv2.imread(img_path)

            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                axes[i].imshow(img_rgb)

                # Title with score and category
                category = retrieval_system.categories[idx]
                axes[i].set_title(f"{category}\nScore: {score:.3f}",
                                fontsize=9)

                # Add border for labeled images
                if idx in retrieval_system.relevant_indices:
                    for spine in axes[i].spines.values():
                        spine.set_edgecolor('green')
                        spine.set_linewidth(4)
                elif idx in retrieval_system.irrelevant_indices:
                    for spine in axes[i].spines.values():
                        spine.set_edgecolor('red')
                        spine.set_linewidth(4)

        axes[i].axis('off')

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"✓ Visualization saved to: {save_path}")

    plt.show()


In [ ]:
def evaluate_system(retrieval_system, query_idx, ground_truth_category,
                   max_rounds=5):
    """Evaluate retrieval performance"""

    # Get all images from same category as ground truth
    ground_truth_indices = [i for i, cat in enumerate(retrieval_system.categories)
                           if cat == ground_truth_category and i != query_idx]

    results_history = {
        'round': [],
        'precision': [],
        'recall': [],
        'relevant_count': [],
        'irrelevant_count': []
    }

    print(f"\n{'='*70}")
    print("EVALUATION MODE")
    print(f"{'='*70}")
    print(f"Query Category: {ground_truth_category}")
    print(f"Ground Truth Size: {len(ground_truth_indices)} relevant images")

    # Round 0
    results, scores = retrieval_system.initial_retrieval(query_idx, top_k=100)
    p = compute_metrics(results[:20], ground_truth_indices)

    results_history['round'].append(0)
    results_history['precision'].append(p['precision'])
    results_history['recall'].append(p['recall'])
    results_history['relevant_count'].append(0)
    results_history['irrelevant_count'].append(0)

    print(f"\nRound 0 - P@20: {p['precision']:.3f}, R@20: {p['recall']:.3f}")

    # Simulated feedback rounds
    for round_num in range(1, max_rounds + 1):
        # Simulate feedback: top-3 relevant, next-3 irrelevant
        relevant_feedback = [idx for idx in results[:10]
                           if idx in ground_truth_indices][:3]
        irrelevant_feedback = [idx for idx in results[:10]
                             if idx not in ground_truth_indices][:3]

        if not relevant_feedback:
            print(f"No relevant images found in top-10. Stopping at round {round_num-1}")
            break

        retrieval_system.add_feedback(relevant_feedback, irrelevant_feedback)
        retrieval_system.train_svm()

        results, scores = retrieval_system.svm_retrieval(top_k=100)
        if results is None:
            break

        p = compute_metrics(results[:20], ground_truth_indices)

        results_history['round'].append(round_num)
        results_history['precision'].append(p['precision'])
        results_history['recall'].append(p['recall'])
        results_history['relevant_count'].append(len(retrieval_system.relevant_indices))
        results_history['irrelevant_count'].append(len(retrieval_system.irrelevant_indices))

        print(f"Round {round_num} - P@20: {p['precision']:.3f}, R@20: {p['recall']:.3f}")

    # Plot results
    plot_evaluation(results_history)

    return results_history

def compute_metrics(retrieved, ground_truth):
    """Compute precision and recall"""
    retrieved_set = set(retrieved)
    gt_set = set(ground_truth)

    hits = len(retrieved_set.intersection(gt_set))

    precision = hits / len(retrieved) if len(retrieved) > 0 else 0
    recall = hits / len(gt_set) if len(gt_set) > 0 else 0

    return {'precision': precision, 'recall': recall, 'hits': hits}

def plot_evaluation(results_history):
    """Plot evaluation metrics"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Precision and Recall
    axes[0].plot(results_history['round'], results_history['precision'],
                marker='o', label='Precision@20', linewidth=2)
    axes[0].plot(results_history['round'], results_history['recall'],
                marker='s', label='Recall@20', linewidth=2)
    axes[0].set_xlabel('Feedback Round', fontsize=12)
    axes[0].set_ylabel('Score', fontsize=12)
    axes[0].set_title('Precision and Recall vs. Feedback Rounds', fontsize=13, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_ylim([0, 1.0])

    # Feedback counts
    axes[1].bar(results_history['round'][1:], results_history['relevant_count'][1:],
               label='Relevant', alpha=0.7, color='green')
    axes[1].bar(results_history['round'][1:], results_history['irrelevant_count'][1:],
               bottom=results_history['relevant_count'][1:],
               label='Irrelevant', alpha=0.7, color='red')
    axes[1].set_xlabel('Feedback Round', fontsize=12)
    axes[1].set_ylabel('Cumulative Feedback Count', fontsize=12)
    axes[1].set_title('Feedback Accumulation', fontsize=13, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.savefig('evaluation_results.png', dpi=150, bbox_inches='tight')
    print("\n✓ Evaluation plot saved: evaluation_results.png")
    plt.show()

In [ ]:
def main():
    """Main entry point"""
    import argparse

    parser = argparse.ArgumentParser(
        description='SVM-Based Image Retrieval with Multi-Round Relevance Feedback'
    )
    parser.add_argument('--dataset', default='cifar10_images',
                       help='Path to image dataset folder')
    parser.add_argument('--features', default='features.npz',
                       help='Path to save/load features')
    parser.add_argument('--extract', action='store_true',
                       help='Force feature extraction even if features file exists')
    parser.add_argument('--query-idx', type=int, default=None,
                       help='Query image index (if not provided, will prompt)')
    parser.add_argument('--mode', choices=['interactive', 'evaluate', 'demo'],
                       default='interactive',
                       help='Running mode')
    parser.add_argument('--max-rounds', type=int, default=5,
                       help='Maximum number of feedback rounds')
    parser.add_argument('--visualize', action='store_true',
                       help='Show visual results (requires matplotlib)')

    args = parser.parse_args([]) # Fix: Pass an empty list to avoid parsing kernel arguments

    print("\n" + "="*70)
    print("SVM-BASED IMAGE RETRIEVAL SYSTEM")
    print("Multi-Round Relevance Feedback")
    print("="*70)

    # Check if dataset exists
    if not os.path.exists(args.dataset):
        print(f"\n❌ Dataset folder not found: {args.dataset}")
        print("\nPlease run: python setup_dataset.py")
        return

    # Load or extract features
    if args.extract or not os.path.exists(args.features):
        print(f"\n📊 Extracting features from: {args.dataset}")
        features, filenames, categories, scaler = extract_dataset_features(
            args.dataset, args.features
        )
    else:
        print(f"\n📊 Loading pre-computed features from: {args.features}")
        data = np.load(args.features, allow_pickle=True)
        features = data['features']
        filenames = data['filenames']
        categories = data['categories']
        print(f"✓ Loaded {len(features)} feature vectors")
        print(f"✓ Feature dimension: {features.shape[1]}")

    # Initialize retrieval system
    retrieval_system = SVMRetrievalSystem(features, filenames, categories,
                                         args.dataset)

    # Get unique categories
    unique_categories = sorted(set(categories))
    print(f"\n📁 Dataset Statistics:")
    print(f"   Total images: {len(filenames)}")
    print(f"   Categories: {len(unique_categories)}")
    for cat in unique_categories:
        count = sum(1 for c in categories if c == cat)
        print(f"      - {cat}: {count} images")

    # Select query image
    if args.query_idx is None:
        print(f"\n🔍 Select Query Image:")
        print(f"   Sample images (first 20):")
        for i in range(min(20, len(filenames))):
            print(f"      [{i:3d}] {categories[i]:12s} - {filenames[i]}")
        print("   ...")

        query_idx = int(input("\nEnter query image index: "))
    else:
        query_idx = args.query_idx

    if query_idx < 0 or query_idx >= len(filenames):
        print(f"❌ Invalid query index: {query_idx}")
        return

    # Run based on mode
    if args.mode == 'interactive':
        run_cli_session(retrieval_system, query_idx, max_rounds=args.max_rounds)

    elif args.mode == 'evaluate':
        query_category = categories[query_idx]
        evaluate_system(retrieval_system, query_idx, query_category,
                       max_rounds=args.max_rounds)

    elif args.mode == 'demo':
        print("\n🎬 Running Demo Mode...")
        results, scores = retrieval_system.initial_retrieval(query_idx, top_k=20)

        if args.visualize:
            visualize_results(retrieval_system, results, scores,
                            "Initial Retrieval Results",
                            save_path="demo_results.png")
        else:
            display_images_cli(retrieval_system, results, scores,
                             "Initial Retrieval Results")

if __name__ == "__main__":
    main()


SVM-BASED IMAGE RETRIEVAL SYSTEM
Multi-Round Relevance Feedback

📊 Loading pre-computed features from: features.npz
✓ Loaded 60000 feature vectors
✓ Feature dimension: 131

📁 Dataset Statistics:
   Total images: 60000
   Categories: 10
      - airplane: 6000 images
      - automobile: 6000 images
      - bird: 6000 images
      - cat: 6000 images
      - deer: 6000 images
      - dog: 6000 images
      - frog: 6000 images
      - horse: 6000 images
      - ship: 6000 images
      - truck: 6000 images

🔍 Select Query Image:
   Sample images (first 20):
      [  0] ship         - ship/test_05104.png
      [  1] ship         - ship/train_11577.png
      [  2] ship         - ship/train_31981.png
      [  3] ship         - ship/train_13912.png
      [  4] ship         - ship/train_33562.png
      [  5] ship         - ship/train_24929.png
      [  6] ship         - ship/train_35236.png
      [  7] ship         - ship/train_19788.png
      [  8] ship         - ship/train_49330.png
      [  9


======================================================================
SVM-BASED IMAGE RETRIEVAL SYSTEM
Multi-Round Relevance Feedback
======================================================================

📊 Loading pre-computed features from: features.npz
✓ Loaded 60000 feature vectors
✓ Feature dimension: 131

📁 Dataset Statistics:
   Total images: 60000
   Categories: 10
      - airplane: 6000 images
      - automobile: 6000 images
      - bird: 6000 images
      - cat: 6000 images
      - deer: 6000 images
      - dog: 6000 images
      - frog: 6000 images
      - horse: 6000 images
      - ship: 6000 images
      - truck: 6000 images

🔍 Select Query Image:
   Sample images (first 20):
      [  0] ship         - ship/test_05104.png
      [  1] ship         - ship/train_11577.png
      [  2] ship         - ship/train_31981.png
      [  3] ship         - ship/train_13912.png
      [  4] ship         - ship/train_33562.png
      [  5] ship         - ship/train_24929.png
      [  6] ship         - ship/train_35236.png
      [  7] ship         - ship/train_19788.png
      [  8] ship         - ship/train_49330.png
      [  9] ship         - ship/train_41028.png
      [ 10] ship         - ship/test_05814.png
      [ 11] ship         - ship/train_07580.png
      [ 12] ship         - ship/train_05806.png
      [ 13] ship         - ship/train_48120.png
      [ 14] ship         - ship/train_38952.png
      [ 15] ship         - ship/train_46048.png
      [ 16] ship         - ship/train_33756.png
      [ 17] ship         - ship/train_41943.png
      [ 18] ship         - ship/train_11309.png
      [ 19] ship         - ship/train_49180.png
   ...

Enter query image index: 20

======================================================================
SVM IMAGE RETRIEVAL - MULTI-ROUND RELEVANCE FEEDBACK
======================================================================

🔍 Query Image: ship/train_46449.png
   Category: ship

──────────────────────────────────────────────────────────────────────
ROUND 0: Initial Retrieval (Distance-based)
──────────────────────────────────────────────────────────────────────

======================================================================
                        Top 20 Initial Results                        
======================================================================
[ 0] Score: -6.7259 | airplane     | airplane/train_35570.png
[ 1] Score: -7.1911 | ship         | ship/train_37659.png
[ 2] Score: -7.1922 | ship         | ship/train_24811.png
[ 3] Score: -7.5685 | automobile   | automobile/train_30088.png
[ 4] Score: -7.5831 | airplane     | airplane/train_08553.png
[ 5] Score: -7.6902 | airplane     | airplane/train_39027.png
[ 6] Score: -7.7072 | bird         | bird/train_36305.png
[ 7] Score: -7.7196 | airplane     | airplane/train_14468.png
[ 8] Score: -7.7423 | ship         | ship/train_28855.png
[ 9] Score: -7.8100 | truck        | truck/train_46611.png
[10] Score: -7.8117 | automobile   | automobile/train_23923.png
[11] Score: -7.8520 | truck        | truck/train_06903.png
[12] Score: -7.8738 | automobile   | automobile/train_31251.png
[13] Score: -7.9254 | dog          | dog/train_47054.png
[14] Score: -7.9261 | automobile   | automobile/train_38958.png
[15] Score: -7.9537 | airplane     | airplane/train_16513.png
[16] Score: -7.9928 | ship         | ship/train_05648.png
[17] Score: -7.9931 | ship         | ship/train_31196.png
[18] Score: -8.0256 | bird         | bird/train_49357.png
[19] Score: -8.0678 | horse        | horse/train_17649.png

──────────────────────────────────────────────────────────────────────
ROUND 1: Provide Feedback
──────────────────────────────────────────────────────────────────────

Enter indices of RELEVANT images (e.g., 0,3,5 or 0-5): 0
Enter indices of IRRELEVANT images (e.g., 1,2,4 or 6-10): 4

📊 Feedback Round 1:
   Relevant: 1 (Total: 1)
   Irrelevant: 1 (Total: 1)

🔧 Training SVM...
🔍 Retrieving with SVM...

======================================================================
                     Top 20 Results (SVM Round 1)                     
======================================================================
[ 0] Score: -0.1665 | airplane     | airplane/train_30297.png
[ 1] Score: -0.1667 | airplane     | airplane/train_11029.png
[ 2] Score: -0.2246 | cat          | cat/train_15412.png
[ 3] Score: -0.2281 | airplane     | airplane/test_03112.png
[ 4] Score: -0.2500 | truck        | truck/test_09927.png
[ 5] Score: -0.2640 | bird         | bird/train_35800.png
[ 6] Score: -0.2673 | airplane     | airplane/train_43998.png
[ 7] Score: -0.2731 | ship         | ship/train_24945.png
[ 8] Score: -0.2734 | deer         | deer/train_22513.png
[ 9] Score: -0.2815 | airplane     | airplane/train_13894.png
[10] Score: -0.2828 | ship         | ship/train_09951.png
[11] Score: -0.2849 | ship         | ship/train_13325.png
[12] Score: -0.2854 | airplane     | airplane/train_39447.png
[13] Score: -0.2891 | ship         | ship/train_43381.png
[14] Score: -0.2909 | ship         | ship/test_08602.png
[15] Score: -0.2921 | ship         | ship/test_03226.png
[16] Score: -0.2924 | deer         | deer/train_01254.png
[17] Score: -0.2934 | airplane     | airplane/train_13473.png
[18] Score: -0.2946 | ship         | ship/train_04466.png
[19] Score: -0.2952 | ship         | ship/train_01699.png

──────────────────────────────────────────────────────────────────────
Continue to Round 2? (y/n): y

──────────────────────────────────────────────────────────────────────
ROUND 2: Provide Feedback
──────────────────────────────────────────────────────────────────────

Enter indices of RELEVANT images (e.g., 0,3,5 or 0-5): 5
Enter indices of IRRELEVANT images (e.g., 1,2,4 or 6-10): 6

📊 Feedback Round 2:
   Relevant: 1 (Total: 2)
   Irrelevant: 1 (Total: 2)

🔧 Training SVM...
🔍 Retrieving with SVM...

======================================================================
                     Top 20 Results (SVM Round 2)                     
======================================================================
[ 0] Score: -0.2361 | airplane     | airplane/test_03112.png
[ 1] Score: -0.2434 | ship         | ship/test_08602.png
[ 2] Score: -0.2550 | airplane     | airplane/train_11029.png
[ 3] Score: -0.2611 | truck        | truck/test_09927.png
[ 4] Score: -0.2759 | frog         | frog/train_46535.png
[ 5] Score: -0.2983 | airplane     | airplane/train_45088.png
[ 6] Score: -0.3005 | airplane     | airplane/train_26296.png
[ 7] Score: -0.3033 | airplane     | airplane/train_25816.png
[ 8] Score: -0.3179 | deer         | deer/train_22513.png
[ 9] Score: -0.3235 | airplane     | airplane/train_38670.png
[10] Score: -0.3253 | ship         | ship/test_08951.png
[11] Score: -0.3283 | ship         | ship/train_11948.png
[12] Score: -0.3320 | deer         | deer/test_04675.png
[13] Score: -0.3320 | airplane     | airplane/test_00975.png
[14] Score: -0.3322 | airplane     | airplane/train_13894.png
[15] Score: -0.3333 | deer         | deer/train_00020.png
[16] Score: -0.3343 | ship         | ship/train_38567.png
[17] Score: -0.3347 | bird         | bird/train_08770.png
[18] Score: -0.3349 | deer         | deer/train_33688.png
[19] Score: -0.3405 | ship         | ship/train_19587.png

──────────────────────────────────────────────────────────────────────
Continue to Round 3? (y/n): n

======================================================================
SESSION COMPLETE
======================================================================

Total Feedback Collected:
  ✓ Relevant images: 2
  ✗ Irrelevant images: 2
  Total rounds: 2
